In [3]:
import networkx as nx
import pandas as pd
import torch
from torch_geometric.data import Data
import numpy as np

def build_network_graph(topology_df):
    """Build a graph from topology_data_logical table"""
    
    G = nx.Graph()
    
    # Add nodes and edges with attributes
    for _, row in topology_df.iterrows():
        aend = row['aendname'].upper()
        bend = row['bendname'].upper()
        
        # Add node attributes
        G.add_node(aend, ip=row['aendip'], 
                   physicalringname=row['physicalringname'], 
                   lrname=row['lrname'],
                   block_name=row['block_name'])
        
        G.add_node(bend, ip=row['bendip'], 
                   physicalringname=row['physicalringname'], 
                   lrname=row['lrname'],
                   block_name=row['block_name'])
        
        # Add edge with attributes
        G.add_edge(aend, bend, 
                  physicalringname=row['physicalringname'], 
                  lrname=row['lrname'],
                  aendifIndex=row['aendifIndex'],
                  bendifIndex=row['bendifIndex'])
    
    return G


In [4]:
def simulate_node_failure(G, node_to_fail):
    """
    Simulate a node failure and identify isolated nodes
    
    Returns:
        - isolated_nodes: list of nodes that become isolated
    """
    # Get the node's ring information
    if node_to_fail not in G.nodes():
        return []
    
    node_pr = G.nodes[node_to_fail]['physicalringname']
    node_lr = G.nodes[node_to_fail]['lrname']
    block_name = G.nodes[node_to_fail]['block_name']
    
    # Get all connected nodes in the same ring before failure
    before_graph = G.copy()
    block_component = set(nx.node_connected_component(before_graph, block_name))
    connected_before = {n for n in block_component 
                        if n in before_graph.nodes() and 
                        before_graph.nodes[n].get('physicalringname') == node_pr and 
                        before_graph.nodes[n].get('lrname') == node_lr}
    
    # Remove the failed node
    after_graph = G.copy()
    after_graph.remove_node(node_to_fail)
    
    # Get connected nodes after failure
    if block_name in after_graph.nodes():
        block_component_after = set(nx.node_connected_component(after_graph, block_name))
        connected_after = {n for n in block_component_after 
                          if n in after_graph.nodes() and 
                          after_graph.nodes[n].get('physicalringname') == node_pr and 
                          after_graph.nodes[n].get('lrname') == node_lr}
    else:
        connected_after = set()
    
    # Find isolated nodes (nodes connected before but not after)
    isolated_nodes = connected_before - connected_after - {node_to_fail}
    
    return list(isolated_nodes)

def simulate_edge_failure(G, edge_to_fail):
    """
    Simulate an edge failure and identify isolated nodes
    
    Returns:
        - isolated_nodes: list of nodes that become isolated
    """
    u, v = edge_to_fail
    if not G.has_edge(u, v):
        return []
    
    # Get edge information
    edge_pr = G.edges[u, v]['physicalringname']
    edge_lr = G.edges[u, v]['lrname']
    block_name = G.nodes[u]['block_name']  # Assuming same block for connected nodes
    
    # Get connected nodes before failure
    before_graph = G.copy()
    block_component = set(nx.node_connected_component(before_graph, block_name))
    connected_before = {n for n in block_component 
                        if n in before_graph.nodes() and 
                        before_graph.nodes[n].get('physicalringname') == edge_pr and 
                        before_graph.nodes[n].get('lrname') == edge_lr}
    
    # Remove the edge
    after_graph = G.copy()
    after_graph.remove_edge(u, v)
    
    # Get connected nodes after failure
    block_component_after = set(nx.node_connected_component(after_graph, block_name))
    connected_after = {n for n in block_component_after 
                       if n in after_graph.nodes() and 
                       after_graph.nodes[n].get('physicalringname') == edge_pr and 
                       after_graph.nodes[n].get('lrname') == edge_lr}
    
    # Find isolated nodes
    isolated_nodes = connected_before - connected_after
    
    return list(isolated_nodes)


In [5]:
def create_gnn_dataset(topology_df, num_simulations=100):
    """
    Create a dataset for training a GNN to predict isolated nodes
    
    Returns:
        - List of PyTorch Geometric Data objects
    """
    # Build the graph
    G = build_network_graph(topology_df)
    
    # Node mapping (for creating numerical indices)
    node_list = list(G.nodes())
    node_to_idx = {node: i for i, node in enumerate(node_list)}
    
    # Create node features
    node_features = []
    for node in node_list:
        # Feature 1: One-hot encoded physical ring
        pr = G.nodes[node]['physicalringname']
        pr_hash = hash(pr) % 10  # Simple encoding
        
        # Feature 2: One-hot encoded logical ring
        lr = G.nodes[node]['lrname']
        lr_hash = hash(lr) % 10  # Simple encoding
        
        # Feature 3: Is it a block node?
        is_block = 1.0 if node == G.nodes[node]['block_name'] else 0.0
        
        # Feature 4-5: Degree centrality and clustering coefficient
        degree = G.degree(node) / len(G)
        clustering = nx.clustering(G, node)
        
        node_features.append([pr_hash/10.0, lr_hash/10.0, is_block, degree, clustering])
    
    node_features = torch.tensor(node_features, dtype=torch.float)
    
    # Create edge index for PyG
    edges = []
    for u, v in G.edges():
        edges.append([node_to_idx[u], node_to_idx[v]])
        edges.append([node_to_idx[v], node_to_idx[u]])  # Add reverse edge for undirected graph
    
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    
    # Create dataset entries
    data_list = []
    
    # Simulate node failures
    for _ in range(num_simulations // 2):  # Half node failures, half edge failures
        # Randomly select a node to fail
        node_to_fail = np.random.choice(node_list)
        
        # Find isolated nodes
        isolated_nodes = simulate_node_failure(G, node_to_fail)
        
        # Create target tensor (1 for isolated nodes, 0 for others)
        y = torch.zeros(len(node_list), dtype=torch.float)
        for node in isolated_nodes:
            y[node_to_idx[node]] = 1.0
        
        # Create node failure mask
        node_mask = torch.zeros(len(node_list), dtype=torch.bool)
        node_mask[node_to_idx[node_to_fail]] = True
        
        # Create PyG Data object
        data = Data(
            x=node_features,
            edge_index=edge_index,
            y=y,
            failed_node_mask=node_mask,
            failed_edge_mask=None
        )
        
        data_list.append(data)
    
    # Simulate edge failures
    edge_list = list(G.edges())
    for _ in range(num_simulations // 2):
        # Randomly select an edge to fail
        edge_to_fail = edge_list[np.random.randint(0, len(edge_list))]
        
        # Find isolated nodes
        isolated_nodes = simulate_edge_failure(G, edge_to_fail)
        
        # Create target tensor
        y = torch.zeros(len(node_list), dtype=torch.float)
        for node in isolated_nodes:
            y[node_to_idx[node]] = 1.0
        
        # Create edge failure mask
        edge_mask = torch.zeros(edge_index.size(1), dtype=torch.bool)
        u_idx, v_idx = node_to_idx[edge_to_fail[0]], node_to_idx[edge_to_fail[1]]
        
        for i in range(edge_index.size(1)):
            e_u, e_v = edge_index[0, i].item(), edge_index[1, i].item()
            if (e_u == u_idx and e_v == v_idx) or (e_u == v_idx and e_v == u_idx):
                edge_mask[i] = True
        
        # Create PyG Data object
        data = Data(
            x=node_features,
            edge_index=edge_index,
            y=y,
            failed_node_mask=None,
            failed_edge_mask=edge_mask
        )
        
        data_list.append(data)
    
    return data_list


In [ ]:
import mysql.connector
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader

def load_topology_data(config):
    """Load topology data from MySQL database"""
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor(dictionary=True)
    
    query = """
        SELECT 
           aendname, 
           bendname, 
           aendip, 
           bendip, 
           aendifIndex,
           bendifIndex,
           block_name, 
           physicalringname, 
           lrname 
        FROM topology_data_logical
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    connection.close()
    
    return pd.DataFrame(rows)


In [11]:


# Database configuration
db_config = {
    "host": "192.168.30.15",
    "user": "nms",
    "password": "Nms@1234",
    "database": "cnmsip"
}

# Load topology data
topology_df = load_topology_data(db_config)

# Create dataset for GNN training
data_list = create_gnn_dataset(topology_df, num_simulations=200)
print(f"Created {len(data_list)} data entries")

KeyError: 'Pakhanjur'